<a href="https://colab.research.google.com/github/wanderson42/Portfolio-DS/blob/main/Projeto_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# K-Nearest Neighbors
***
- Um dos algoritmos de aprendizado de máquina mais fáceis de implementar
- Algoritmo de classificação que faz previsões com base em um número definido de vizinhos mais próximos
    - Tem somente um único hiperparâmetro
- Não requer nenhum "aprendizado" - apenas um simples cálculo de distância
- Ideia básica: uma amostra é classificada por maioria de votos de seus vizinhos mais próximos
- O algoritmo classifica uma instância (elemento do dataset de treino) com base na  similaridade entre K instâncias mais próximas entre si. Portando, o parâmetro de similaridade é a distância (daí que vem o nome KNN). 
- Dicas:
    - Como é um algoritmo baseado em distância, então é uma boa ideia padronizar seus dados antes do treino


## Background teórico

- KNN se resume a uma única fórmula para distância
- Você pode escolher entre várias métricas de distância, como cosseno ou distância euclidiana
- Usarei a distância euclidiana
- Aqui está a fórmula:

$$ \Large d(p, q) = \sqrt{(p_1 - q_1)^2 + (p_2 - q_2)^2 + ... + (p_n - q_n)^2} $$

- Ou de forma mais compacta:

$$ \Large d(p, q) = \sqrt{\sum_{i=1}^n (p_i - q_i)^2} $$

- É uma distância em linha reta simples e fácil de implementar em Python

<br><br>

## Implementação
- A classe `KNN` foi escrita tendo em mente a sintaxe da API Scikit-Learn
- O valor de `k` é passado como parâmetro no método construtor (`__init__()`) ao lado de `X_train` e `y_train`, inicialmente definido como `None`
- O `_euclidean_distance(p, q)` é um método privado que implementa a fórmula acima
- O método `fit(X, y)` não faz basicamente nada - apenas armazena dados de treinamento para o construtor
- O método `predict(X)` calcula as distâncias entre cada linha em `X` e cada linha em `X_train`. As distâncias são então ordenadas de forma crescente e apenas o k que correspondem às menores distâncias é mantido. A classificação é feita através da função scipy stats.mode()

In [36]:
from scipy import stats

class TuringKNN:
    '''
    Uma classe escrita ao estilo "from scratch" que implementa
    o algoritmo k-Nearest Neighbors.    
    '''
    def __init__(self, k):
        self.k = k
        self.X_train = None
        self.y_train = None
        
    @staticmethod
    def _euclidean_distance(p, q):
        '''
        Método privado para calcular a distância euclidiana entre dois vetores.
        
        :param p: np.array, 1º vetor
        :param q: np.array, 2º vetor
        :return: float, distância
        '''
        return np.sqrt(np.sum((p - q) ** 2))
        
    def fit(self, X, y):
        '''
        Nenhum treinamento é necessário para KNN, então 'fit(X, y)' salva ambos os parâmetros
        no método construtor.
        
        :param X: pd.DataFrame, features
        :param y: pd.Series, target
        :return: None
        '''
        self.X_train = X
        self.y_train = y
        
    def predict(self, X):
        '''
        Efetua as predictions baseado nos vizinhos mais próximos.
        
        :param X: pd.DataFrame, features
        :return: np.array, predictions
        '''
        predictions = []
        for p in X:
            euc_distances = [self._euclidean_distance(p, q) for q in self.X_train]
            sorted_k = np.argsort(euc_distances)[:self.k]
            k_nearest = [self.y_train[y] for y in sorted_k]
            predictions.append(stats.mode(k_nearest)[0][0])
            
        return np.array(predictions)

<br>

## Teste
- Eu utilizei o *breast cancer* dataset do Scikit-Learn:

In [37]:
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
X = data.data
y = data.target 

- O código abaixo aplica a divisão de treinamento/teste ao dataset:

In [38]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

- Agora pode-se inicializar e "treinar" o modelo e depois fazer as previsões:

In [39]:
model = TuringKNN(3)
model.fit(X_train, y_train)
preds = model.predict(X_test)

In [40]:
preds

array([0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1,
       0, 1, 1, 0])

In [41]:
y_test

array([1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1,
       0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 1, 0])

- Como podemos ver, os arrays são bastante semelhantes, mas têm algumas diferenças
- Vamos calcular a precisão para avaliar o modelo:

In [42]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, preds)

0.9298245614035088

- Conforme a acurácia de $≈93 \%$ , o algoritmo KNN "from scratch" teve uma excelente performance  
- Vamos ajustá-lo encontrando o valor K ideal

<br>

## Otimização de K e Comparação com Scikit-Learn
- Queremos saber se o modelo TuringKNN é realmente bom, então vamos compará-lo com o modelo `KNeighborsClassifier` do Scikit-Learn
- O valor K de 3 talvez não seja a melhor escolha
- Vamos treinar e avaliar ambos os modelos para cada número ímpar de 1 a 15
- Os resultados são armazenados nos Dataframe's `evals_turing` e `evals_sklearn`:

In [43]:
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier

evals_turing  = []
evals_sklearn = []

for k in range(1, 16, 2):
    '''  
    Otimização de K via algoritmo TuringKnn
    '''
    model = TuringKNN(k=k)
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    accuracy = accuracy_score(y_test, preds)
    evals_turing.append({'k': k, 'accuracy': accuracy})
    '''
    Otimização de K via algoritmo KNeighborsClassifier do Sklearn
    '''
    knn_model = KNeighborsClassifier(k)
    knn_model.fit(X_train, y_train)
    knn_preds = knn_model.predict(X_test)
    accuracy = accuracy_score(y_test, knn_preds)
    evals_sklearn.append({'k': k, 'accuracy': accuracy})

evals_turing = pd.DataFrame(evals_turing)
evals_sklearn = pd.DataFrame(evals_sklearn)

- Vamos inspecionar visualmente o melhor valor de K em ambos os modelos:

In [44]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go

fig = make_subplots(rows=1, cols=2)

fig.add_scatter(x = evals_turing['k'], y = evals_turing['accuracy'],
                name="TuringKNN", row=1, col=1)

fig.add_scatter(x = evals_sklearn['k'], y = evals_sklearn['accuracy'],
                name="SklearnKNN", row=1, col=2)

# Update xaxis properties
fig.update_xaxes(title_text="K", row=1, col=1, range=[0, 16])
fig.update_xaxes(title_text="K", row=1, col=2, range=[0, 16])

# Update yaxis properties
fig.update_yaxes(title_text="Acurácia", row=1, col=1)
fig.update_yaxes(title_text="Acurácia", row=1, col=2)

fig.show()

- Com o dataset *breast cancer* o valor K = 11 corresponde aos melhores resultados, obtendo-se uma acurácia superior de $≈98 \%$

